In [82]:
import feedparser
import time

In [67]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re

driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
extension_path = 'C:\\chromedriver-win64\\bypass-paywalls-chrome-clean-master.crx'

service = Service(executable_path=driver_path)
option = Options()
option.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36')
option.add_argument("--disable-blink-features=AutomationControlled")
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_experimental_option('useAutomationExtension', False)
option.add_extension(extension=extension_path)
web = webdriver.Chrome(service=service,options=option)

In [83]:
def webGet(url, title, class_list, label):
    web.get(url=url)
    time.sleep(3)
    if(label == 'class'):
        css_selector = "." + ", .".join(class_list)
    elif(label == 'p'):
        css_selector = "p"
    textdata = web.find_elements(By.CSS_SELECTOR, css_selector)
    artifact = ''
    for data in textdata:
        artifact = artifact + data.text + '\n'
    safe_filename = re.sub(r'[<>:"/\\|?*]', '_', title) + ".txt"
    with open(safe_filename,"w") as file:
        file.write(artifact)

In [62]:
web.get(url='https://pixelscan.net/')

In [57]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [63]:
button = WebDriverWait(web, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'START')]"))
)

# 点击按钮
button.click()

In [88]:
def rssGet(url, topN=5, i=0):
  feed = feedparser.parse(url)
  print(f"Feed Title: {feed.feed.title}")
  print(f"Feed Description: {getattr(feed.feed, 'description', 'unknown')}")
  for entry in feed.entries[:topN]:
    print("Title:", entry.title)
    print("Link:", entry.link)
    print("Published:", entry.published)
    print("-" * 80)
  output = {
    'title':getattr(feed.entries[i], 'title', 'unknown'),
    'url':feed.entries[i].link,
    'author':getattr(feed.entries[i], 'author', 'unknown'),
    'date':feed.entries[i].published
  }
  return output

In [91]:
news = rssGet('https://www.scmp.com/rss/5/feed',3,0)

Feed Title: World - South China Morning Post
Feed Description: Breaking news, analysis and opinion from the SCMP's international edition.
Title: Carney’s Liberals win Canada election dominated by Trump
Link: https://www.scmp.com/news/us/diplomacy/article/3308287/carneys-liberals-win-canada-election-according-cbc-projection?utm_source=rss_feed
Published: Tue, 29 Apr 2025 10:31:44 +0800
--------------------------------------------------------------------------------
Title: How China is turning Trump’s tariffs into an opportunity
Link: https://www.scmp.com/opinion/china-opinion/article/3308096/how-china-turning-trumps-tariffs-opportunity?utm_source=rss_feed
Published: Tue, 29 Apr 2025 09:30:08 +0800
--------------------------------------------------------------------------------
Title: Amnesty accuses Israel of ‘live-streamed genocide’ against Gaza Palestinians
Link: https://www.scmp.com/news/world/europe/article/3308279/amnesty-accuses-israel-live-streamed-genocide-against-gaza-palestini

In [92]:
webGet(news['url'],news['title'],['paragraph-base'],'p')